In [7]:
import pandas as pd
import glob
import os
from scipy.stats import zscore
from utils import find_nulls, add_ratio_features, add_z_score

In [8]:
PATH_SAMPLE = '../dataset/real-time/test_after_attack/after_attack_barley.csv'
CSV_FOLDER = '../dataset/real-time/test_after_attack'
CSV_PREPROCESSED_FOLDER = '../dataset/real-time/test_after_attack_preprocessed'

Z_SCORE_COLS =  ['trace_involved_amt','contract_block_involved','contract_tx_count','contract_main_active_days',
                'sender_block_involved','sender_tx_count','sender_main_active_days','contract_interact',
                'sender_tx_count_call_contract','sender_days_call_contract','trace_amt','distinct_sender_in_contract',
                'contract_lifetime_days','contract_lifetime_block','distinct_contract_sender_called',
                'sender_lifetime_days','sender_lifetime_block','contract_involved_amt','max_breadth','depth',
                'distinct_was_called_in_sample','distinct_sender_call_in_sample','gas','gas_price',
                'receipt_cumulative_gas_used','receipt_gas_used','value','nonce']

SUS_FILE = '../dataset/sus_tx.csv'  # Replace with the actual file path

In [9]:
def impute_depth_and_max_breadth(path):
    df = pd.read_csv(path)
    df['max_breadth'] = df['max_breadth']+1
    df['max_breadth'] = df['max_breadth'].fillna(0)
    df['depth'] = df['depth'].fillna(0)
    return df

def add_sus_col(df,attack_transactions):

    def set_value_based_on_condition(row):
        if (row['transaction_hash'] in attack_transactions):
            return 1
        else:
            return 0
        
    df['is_sus'] = df.apply(set_value_based_on_condition, axis=1)

    sus_data = df[df['is_sus'] == 1]
    not_sus_data = df[df['is_sus'] == 0]

    print(len(sus_data))
    print(len(not_sus_data))

    return df

In [10]:
sample_df = pd.read_csv(PATH_SAMPLE)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 37 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   transaction_hash                 93 non-null     object 
 1   trace_involved_amt               93 non-null     int64  
 2   from_address                     93 non-null     object 
 3   to_address                       93 non-null     object 
 4   block_timestamp                  93 non-null     object 
 5   gas                              93 non-null     int64  
 6   gas_price                        93 non-null     int64  
 7   receipt_cumulative_gas_used      93 non-null     int64  
 8   receipt_gas_used                 93 non-null     int64  
 9   value                            93 non-null     int64  
 10  nonce                            93 non-null     int64  
 11  transaction_type                 93 non-null     int64  
 12  rn                      

In [11]:
for file_path in glob.glob(os.path.join(CSV_FOLDER, "*.csv")):
    df = pd.read_csv(file_path)
    find_nulls(df)

max_breadth    12
depth          12
dtype: int64
max_breadth    1
depth          1
dtype: int64
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
max_breadth    29
depth          29
dtype: int64
Series([], dtype: int64)
Series([], dtype: int64)


In [12]:
for file_path in glob.glob(os.path.join(CSV_FOLDER, "*.csv")):
    filename = os.path.basename(file_path)
    print(filename)
    preprocessed_file_path = CSV_PREPROCESSED_FOLDER+'/'+filename
    imputed_df = impute_depth_and_max_breadth(file_path)

    imputed_df['value'] = imputed_df['value'].astype('float32')

    z_score_df = add_z_score(imputed_df,Z_SCORE_COLS)

    ratio_df = add_ratio_features(z_score_df)

    sus_df = pd.read_csv(SUS_FILE)
    sus_transactions = sus_df['sus_tx']
    sus_transactions = set(sus_transactions)
    added_sus_df = add_sus_col(ratio_df,sus_transactions)

    print('Preprocessed file:',filename)
    find_nulls(z_score_df)

    added_sus_df.to_csv(preprocessed_file_path,index=False)
    

after_attack_barley.csv
0
93
Preprocessed file: after_attack_barley.csv
Series([], dtype: int64)
after_attack_chainpaint.csv
0
440
Preprocessed file: after_attack_chainpaint.csv
Series([], dtype: int64)
after_attack_curve.csv
0
52
Preprocessed file: after_attack_curve.csv
Series([], dtype: int64)
after_attack_earningfarm.csv
0
8
Preprocessed file: after_attack_earningfarm.csv
Series([], dtype: int64)
after_attack_gooddollar.csv
0
17
Preprocessed file: after_attack_gooddollar.csv
Series([], dtype: int64)
after_attack_nfttrader.csv
0
35
Preprocessed file: after_attack_nfttrader.csv
Series([], dtype: int64)
after_attack_peapods.csv
0
177
Preprocessed file: after_attack_peapods.csv
Series([], dtype: int64)
after_attack_rugged.csv
0
68
Preprocessed file: after_attack_rugged.csv
Series([], dtype: int64)
after_attack_sturdy.csv
0
370
Preprocessed file: after_attack_sturdy.csv
Series([], dtype: int64)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
